# Problem 4

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.ticker as ticker   

from P4_bfs import *

# setup spark
conf = SparkConf().setAppName('Graph Processing')
sc = SparkContext(conf=conf)

In [15]:
from P4_bfs import *

import pandas as pd

In [16]:
comicdf = pd.read_csv('source.csv', names=['Character', 'ComicIssue'])
comicdf.groupby('ComicIssue')

In [17]:
keys = comicdf['Character'].unique()

In [18]:
values = range(1, len(keys)+1) # start indices with 1

In [19]:
characterDict = dict(zip(keys, values))

In [20]:
characterDict['FROST, CARMILLA']

1

In [21]:
# create graph by adjacency list (there is a connection between
# two characters if they appear in the same comic issue)

# therefore join the comicdf on comic issue!
mergeddf = pd.merge(comicdf, comicdf, how='inner', on='ComicIssue')

# now remove ComicIssue & all rows with Character_x == Character_y
filtereddf = mergeddf.drop('ComicIssue', 1)
filtereddf = filtereddf[filtereddf['Character_x'] != filtereddf['Character_y']]

In [22]:
# transform string to integers for performance reasons
edgedf = filtereddf.applymap(lambda x: characterDict[x])

In [23]:
import csv

# save the edge list and the dictionary as two separate csv's
edgedf.to_csv('edge_list.csv', header=False, index=False)

writer = csv.writer(open('characters.csv', 'wb'))
entries = sorted(characterDict.items(), key=lambda x: x[1]);
for key, value in entries:
    writer.writerow([value, key]) # flip it (so the vertex index is now the key)

In [24]:
# load vertex dictionary (is basically the character dictionary inverted)
reader = csv.reader(open('characters.csv', 'rb'))
vertexDict = dict(reader)

In [25]:
# this function prepares the rdd
def prepare_rdd(filename):
    rdd = sc.textFile(filename)

    # map string to tuples
    rdd = rdd.map(lambda x: x.split(','))
    rdd = rdd.map(lambda x: (int(x[0]), int(x[1])))
    
    # now group s.t. we have for each vertex an adjacency list of nodes
    rdd = rdd.groupByKey().map(lambda x: (x[0], list(x[1])))
    
    return rdd

In [34]:
# test code
filename = 'edge_list.csv'#'edge_list_simple.csv' # 'edge_list.csv'

rdd = prepare_rdd(filename)
#rdd.take(5)
v0 = [characterDict['CAPTAIN AMERICA'], characterDict['MISS THING/MARY'], characterDict['ORWELL']]



In [38]:
for i in range(0, len(v0)):
    rdd = prepare_rdd(filename)
    num_visited_nodes, rdd = sparkBFS(sc, rdd, v0[i])
    
    print('%s : %d nodes visited' % (vertexDict[str(v0[i])], num_visited_nodes))

CAPTAIN AMERICA : 6407 nodes visited
CAPTAIN AMERICA : 6 nodes visited
CAPTAIN AMERICA : 8 nodes visited


In [37]:
vertexDict[str(v0[0])]

'CAPTAIN AMERICA'

In [28]:
num_visited_nodes

6407